In [1]:
!pip install pandas scikit-learn

In [ ]:
!unzip adult.csv.zip

Archive:  adult.csv.zip
replace adult.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [6]:
import pandas as pd
df = pd.read_csv('adult.csv')
df.head()

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [8]:
col_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship',
             'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

df.columns = col_names

df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income'],
      dtype='object')

In [9]:
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [13]:
df = df[['age', 'workclass', 'marital_status']]
df.head()

,age,workclass,marital_status
0,50,Self-emp-not-inc,Married-civ-spouse
1,38,Private,Divorced
2,53,Private,Married-civ-spouse
3,28,Private,Married-civ-spouse
4,37,Private,Married-civ-spouse


In [19]:
df.workclass.unique()

array([' Self-emp-not-inc', ' Private', ' State-gov', ' Federal-gov',
       ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay',
       ' Never-worked'], dtype=object)

In [21]:
import sagemaker
sess = sagemaker.Session()
uri = sess.upload_data(path='./new.csv')
print(uri)

s3://sagemaker-us-east-1-179201351861/data/new.csv


In [22]:
import boto3
from sagemaker import get_execution_role

role = get_execution_role()

In [23]:
df = df.dropna()
df.head()

,age,workclass,marital_status
0,50,Self-emp-not-inc,Married-civ-spouse
1,38,Private,Divorced
2,53,Private,Married-civ-spouse
3,28,Private,Married-civ-spouse
4,37,Private,Married-civ-spouse


In [50]:
X = df[['workclass']]
y = df[['marital_status']]

In [51]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [52]:
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)


In [53]:
bucket = sagemaker.Session().default_bucket()
boto3.Session().resource('s3').Bucket(bucket).Object('./train_data.csv').upload_file('train_data.csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [54]:
bucket

'sagemaker-us-east-1-179201351861'

In [55]:
boto3.Session().resource('s3').Bucket(bucket).Object('./test_data.csv').upload_file('test_data.csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [56]:
from sagemaker import image_uris
container = image_uris.retrieve('knn',region='us-east-1',version='latest')

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [57]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://sagemaker-us-east-1-179201351861/./train_data.csv',content_type="text/csv",)

In [58]:
s3_input_train

In [59]:
bucket

'sagemaker-us-east-1-179201351861'

In [60]:
sess = sagemaker.Session()
knn = sagemaker.estimator.Estimator(
        container,
        get_execution_role(),
        instance_count=1,
        instance_type="ml.m5.2xlarge",
        output_path='s3://{}/output'.format(bucket),
        sagemaker_session=sess,
    )

In [61]:
knn.set_hyperparameters(k=10,predictor_type='classifier',sample_size=200)
knn.fit({"train": s3_input_train})

INFO:sagemaker:Creating training-job with name: knn-2024-05-30-13-48-35-045


2024-05-30 13:48:35 Starting - Starting the training job...
2024-05-30 13:48:52 Starting - Preparing the instances for training...
2024-05-30 13:49:32 Downloading - Downloading the training image..................
2024-05-30 13:52:28 Training - Training image download completed. Training in progress....Docker entrypoint called with argument(s): train
Running default environment configuration script
[05/30/2024 13:52:55 INFO 139977154246464] Reading default configuration from /opt/amazon/lib/python3.9/site-packages/algorithm/resources/default-conf.json: {'_kvstore': 'dist_async', '_log_level': 'info', '_num_gpus': 'auto', '_num_kv_servers': '1', '_tuning_objective_metric': '', '_faiss_index_nprobe': '5', 'epochs': '1', 'feature_dim': 'auto', 'faiss_index_ivf_nlists': 'auto', 'index_metric': 'L2', 'index_type': 'faiss.Flat', 'mini_batch_size': '5000', '_enable_profiler': 'false'}
[05/30/2024 13:52:55 INFO 139977154246464] Merging with provided configuration from /opt/ml/input/config/hype

UnexpectedStatusException: Error for Training job knn-2024-05-30-13-48-35-045: Failed. Reason: ClientError: Unable to parse csv: rows 1-5000, file /opt/ml/input/data/train/train_data.csv (caused by ValueError)

Caused by: could not convert string to float: 'workclass', exit code: 2

In [62]:
df_train_data = df_train_data[df_train_data.ne('?').all(axis=1)]

# Reset the index after removing rows
df_train_data.reset_index(drop=True, inplace=True)


In [63]:
df_train_data.to_csv('train_data.csv')

In [67]:
# Check for '?' values in all relevant columns and remove corresponding rows
df_train_data = df_train_data[(df_train_data['workclass'] != '?') & 
                              (df_train_data['marital_status'] != '?')]

# Verify unique values again
unique_workclasses = df_train_data['workclass'].unique()
print(unique_workclasses)


[' Private' ' State-gov' ' Self-emp-not-inc' ' Local-gov' ' Self-emp-inc'
 ' ?' ' Federal-gov' ' Without-pay' ' Never-worked']


In [72]:
df_train_data['workclass'].replace({'?':'NaN'},inplace=True)